### dataset preprocessing

```python
import torch 
from torch.utils.data import Dataset,DataLoader

class MyDataset(Dataset):
    def __init__(self,txt,tokenizer,max_length,stride):
        # txt: raw text input
        # tokenizer: tiktoken tokenizer
        # max_length: maximum length of text in one window
        # stride: stride of shifting

        self.input_ids = [] # stores all input ids
        self.target_ids = [] # stores all output ids
        # input ids and target ids has 1 shifting

        token_ids = tokenizer.encode(txt,allowed_special={"<|endoftext>"}) #tokenize entire input text

        for i in range(0,len(token_ids)-max_length,stride):
            input_chunk = token_ids[i:i+max_length] # move windows to get input ids
            target_chunk = token_ids[i+1:i+max_length+1]    # move 1 to right to get target ids

            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))
            
    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self,idx):
        return self.input_ids[idx],self.target_ids[idx]
```

### model configuration
set model config
```python
GPT_CONFIG_124M = {
    "vocab_size": 50257,    # Vocabulary size
    "context_length": 1024, # maximum context length of GPT model
    "emb_dim": 768,         # Word Embedding Vector Dimension
    "n_heads": 12,          # Number of attention heads
    "n_layers": 12,         # Number of decoder blocks
    "drop_rate": 0.1,       # Dropout rate
    "qkv_bias": False       # Query-Key-Value bias
}
```

### LayerNorm
Normalize layer output with mean = 1 and std = 0
```python
class LayerNorm(nn.Module):
    def __init__(self, emb_dim):
        super().__init__()
        self.eps = 1e-5 # avoid dividing zero
        self.scale = nn.Parameter(torch.ones(emb_dim)) # add trainable scaling
        self.shift = nn.Parameter(torch.zeros(emb_dim)) # add trainable shifting

    def forward(self, x):
        mean = x.mean(dim=-1, keepdim=True) # get mean of current layer
        var = x.var(dim=-1, keepdim=True, unbiased=False)   # get std of current layer
        norm_x = (x - mean) / torch.sqrt(var + self.eps)    # calculate layernorm result
        return self.scale * norm_x + self.shift # adding scale and shift
```

### GeLU Activation
A modification of ReLU Activation
```python
class GELU(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return 0.5 * x * (1 + torch.tanh(
            torch.sqrt(torch.tensor(2.0 / torch.pi)) * 
            (x + 0.044715 * torch.pow(x, 3))
        ))
```

### FeedForwardLayer
```python
class FeedForward(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(cfg["emb_dim"], 4 * cfg["emb_dim"]),
            GELU(),
            nn.Linear(4 * cfg["emb_dim"], cfg["emb_dim"]),
        )

    def forward(self, x):
        return self.layers(x)
```

### MultiHeadAttention
```python
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert (d_out % num_heads == 0), \
            "d_out must be divisible by num_heads"

        self.d_out = d_out # output dimension
        self.num_heads = num_heads  # number of heads
        self.head_dim = d_out // num_heads # Reduce the projection dim to match desired output dim

        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias) # Wq matrix
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)  # Wk matrix
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias) # Wv matrix
        self.out_proj = nn.Linear(d_out, d_out)  # Linear layer to combine head outputs
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length, context_length),
                       diagonal=1)
        )

    def forward(self, x):
        b, num_tokens, d_in = x.shape

        keys = self.W_key(x) # Shape: (b, num_tokens, d_out)
        queries = self.W_query(x)   # Shape: (b, num_tokens, d_out)
        values = self.W_value(x)    # Shape: (b, num_tokens, d_out)

        # We implicitly split the matrix by adding a `num_heads` dimension
        # Unroll last dim: (b, num_tokens, d_out) -> (b, num_tokens, num_heads, head_dim)

        #拆解最后一个dimension，使其满足n个头，但是减少最后一个dimension(word embedding dimension)
        # shape: (b, num_tokens,num_heads,head_dim)
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim) 
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)
        # 通过以上拆解，n个头的结果concate之后的dimension与原word embedding dimension结果一致

        # Transpose: (b, num_tokens, num_heads, head_dim) -> (b, num_heads, num_tokens, head_dim)
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        # queries shape:              (b, num_heads, num_tokens, head_dim)
        # keys.transpose(2,3) shape: (b, num_heads, head_dim, num_tokens)
        attn_scores = queries @ keys.transpose(2, 3)  # Dot product for each head
        # attn_scores shape: (b, num_tokens, num_tokens)   

        # Original mask truncated to the number of tokens and converted to boolean
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]

        # Use the mask to fill attention scores
        attn_scores.masked_fill_(mask_bool, -torch.inf)
        
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        # attn_weights shape: (b, num_heads, num_tokens,num_tokens) 
        # values shape: (b, num_heads, num_tokens, head_dim)
        context_vec = (attn_weights @ values).transpose(1, 2) 
        # context_vec shape: (b, num_tokens, num_heads, head_dim)
        
        # Combine heads, where self.d_out = self.num_heads * self.head_dim
        # context_vec shape: (b, num_tokens, out_dim)
        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
        
        context_vec = self.out_proj(context_vec) # optional projection

        return context_vec
```

### Decoder Block
```python

class TransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.att = MultiHeadAttention(
            d_in=cfg["emb_dim"],
            d_out=cfg["emb_dim"],
            context_length=cfg["context_length"],
            num_heads=cfg["n_heads"], 
            dropout=cfg["drop_rate"],
            qkv_bias=cfg["qkv_bias"])

        self.ff = FeedForward(cfg) # feed forward layer
        self.norm1 = LayerNorm(cfg["emb_dim"])
        self.norm2 = LayerNorm(cfg["emb_dim"])
        self.drop_shortcut = nn.Dropout(cfg["drop_rate"])

    def forward(self, x):
        # Shortcut connection for attention block
        shortcut = x    # store og x as shortcut
        x = self.norm1(x) # layer norm input x
        x = self.att(x)  # Shape [batch_size, num_tokens, emb_size] multihead attention layer
        x = self.drop_shortcut(x) # dropout
        #print(x.shape)
        x = x + shortcut  # Add the original input back, add residual

        # Shortcut connection for feed forward block
        shortcut = x #store temp shortcut value
        x = self.norm2(x) # layer norm x
        x = self.ff(x) #feedforward layer
        x = self.drop_shortcut(x) #dropout layer
        x = x + shortcut  # Add the original input back

        return x
```

### Final GPT Model
```python
class GPTModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        self.drop_emb = nn.Dropout(cfg["drop_rate"])
        
        self.trf_blocks = nn.Sequential(
            *[TransformerBlock(cfg) for _ in range(cfg["n_layers"])])
        
        self.final_norm = LayerNorm(cfg["emb_dim"])
        self.out_head = nn.Linear(
            cfg["emb_dim"], cfg["vocab_size"], bias=False
        )

    def forward(self, in_idx):
        batch_size, seq_len = in_idx.shape
        tok_embeds = self.tok_emb(in_idx)
        pos_embeds = self.pos_emb(torch.arange(seq_len, device=in_idx.device))
        x = tok_embeds + pos_embeds  # Shape [batch_size, num_tokens, emb_size]
        x = self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)
        return logits
```
### Output Generation
```python
def generate_text_simple(model, idx, max_new_tokens, context_size):
    # idx is (batch, n_tokens) array of indices in the current context
    for _ in range(max_new_tokens):
        
        # Crop current context if it exceeds the supported context size
        # E.g., if LLM supports only 5 tokens, and the context size is 10
        # then only the last 5 tokens are used as context
        idx_cond = idx[:, -context_size:]
        
        # Get the predictions
        with torch.no_grad():
            logits = model(idx_cond)
        
        # Focus only on the last time step
        # (batch, n_tokens, vocab_size) becomes (batch, vocab_size)
        logits = logits[:, -1, :]  

        # Apply softmax to get probabilities
        probas = torch.softmax(logits, dim=-1)  # (batch, vocab_size)

        # Get the idx of the vocab entry with the highest probability value
        idx_next = torch.argmax(probas, dim=-1, keepdim=True)  # (batch, 1)

        # Append sampled index to the running sequence
        idx = torch.cat((idx, idx_next), dim=1)  # (batch, n_tokens+1)

    return idx
```

In [33]:
import torch
import torch.nn as nn


x = torch.rand((2,2,3,4))
y = torch.rand((2,2,4,3))
z = x @ y
z.shape

torch.Size([2, 2, 3, 3])